In [3]:
import torch.functional as F
import torch
from torch.nn import functional as F 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
ebd = torch.nn.Embedding(10, 4) # 10 words in vocab, 4 dimensional embeddings

words_index = torch.LongTensor([0, 3, 9])

words_ebd = ebd(words_index)

print(words_ebd)

tensor([[ 0.7728, -0.7551,  0.3917,  0.5440],
        [ 1.0195, -0.2443, -0.5720,  2.9042],
        [ 0.7307, -0.9481,  1.0065,  1.0238]], grad_fn=<EmbeddingBackward0>)


In [9]:
q_lin = torch.nn.Linear(4, 4)
q = q_lin(words_ebd)

print(q)

tensor([[-0.2403, -0.0713,  0.6862,  0.4640],
        [ 0.0478, -0.2400,  1.5110,  1.5108],
        [-0.5272, -0.3186,  1.1340,  0.6403]], grad_fn=<AddmmBackward0>)


In [3]:
# a attention demo code
def attention(q, k, v):
    #q, k, v shape: (batch_size, seq_len, dim)
    score = q @ k.transpose(1, 2)
    score = score / (k.size(-1) ** 0.5)
    score = F.softmax(score, dim=-1)
    output = score @ v
    
    return output

In [4]:
batch_size = 1
seq_len = 5
dim = 4

q = torch.randn(batch_size, seq_len, dim)
k = torch.randn(batch_size, seq_len, dim)
v = torch.randn(batch_size, seq_len, dim)

output = attention(q, k, v)
print(output)

tensor([[[-0.3414,  1.0665,  0.0343,  0.0175],
         [-0.4733,  0.9483,  0.7029,  0.4372],
         [-0.2108,  0.9628,  0.5640, -0.3139],
         [-0.6945,  0.2593,  0.9004,  0.6061],
         [-0.2245,  0.1200,  0.4361,  0.9039]]])


In [6]:
# use the attention function, comparing the result with F.scale_dot_product_attention
def test_attention():
    batch_size = 1
    seq_len = 5
    dim = 4
    
    q = torch.randn(batch_size, seq_len, dim)
    k = torch.randn(batch_size, seq_len, dim)
    v = torch.randn(batch_size, seq_len, dim)
    
    
    output = attention(q, k, v)
    print(output)
    
    output2 = F.scaled_dot_product_attention(q, k, v)
    print(output2)
    
    assert torch.allclose(output, output2)
    
test_attention()

tensor([[[ 0.3280, -1.2238,  0.7824,  1.0099],
         [-0.1306,  0.0089, -0.4798, -0.0866],
         [-0.2321,  0.0260, -0.4015, -0.5406],
         [-0.0350, -0.0666, -0.6470,  0.2620],
         [ 0.0600, -0.1234, -0.8371,  0.4399]]])
tensor([[[ 0.3280, -1.2238,  0.7824,  1.0099],
         [-0.1306,  0.0089, -0.4798, -0.0866],
         [-0.2321,  0.0260, -0.4015, -0.5406],
         [-0.0350, -0.0666, -0.6470,  0.2620],
         [ 0.0600, -0.1234, -0.8371,  0.4399]]])
